In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os

In [2]:
wd = '/home/bay001/projects/parp13_ago2_20171015/permanent_data/eCLIP-0.1.7/01_regionlevel_enrichment/outputs/'
rep1 = pd.read_table(os.path.join(wd,'WTSS1.l2fc.txt'), index_col=0).replace('NaN',np.nan)
rep2 = pd.read_table(os.path.join(wd,'WTSS2.l2fc.txt'), index_col=0).replace('NaN',np.nan)
rep1.head()

,rep_01|CDS,rep_01|5utr,rep_01|3utr,rep_01|5utr|3utr,rep_01|intron,rep_01|intergenic,rep_01|noncoding_exon,rep_01|noncoding_intron
ENSG,,,,,,,,
ENSG00000225193.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000254681.2,NaN,NaN,NaN,NaN,NaN,NaN,3.1112,1.6291
ENSG00000228477.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000272370.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000159733.9,0.6962,NaN,NaN,NaN,0.9355,NaN,NaN,NaN


In [3]:
merged = pd.merge(rep1, rep2, how='outer', left_index=True, right_index=True)
merged.head()

,rep_01|CDS_x,rep_01|5utr_x,rep_01|3utr_x,rep_01|5utr|3utr_x,rep_01|intron_x,rep_01|intergenic_x,rep_01|noncoding_exon_x,rep_01|noncoding_intron_x,rep_01|CDS_y,rep_01|5utr_y,rep_01|3utr_y,rep_01|5utr|3utr_y,rep_01|intron_y,rep_01|intergenic_y,rep_01|noncoding_exon_y,rep_01|noncoding_intron_y
ENSG,,,,,,,,,,,,,,,,
ENSG00000000003.10,3.6427,NaN,2.3706,NaN,0.6804,NaN,NaN,NaN,-3.2836,NaN,-1.9341,NaN,-0.8961,NaN,NaN,NaN
ENSG00000000005.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000000419.8,0.7458,NaN,2.6962,NaN,-0.6479,NaN,NaN,NaN,-1.7559,NaN,-1.5336,NaN,0.6446,NaN,NaN,NaN
ENSG00000000457.9,NaN,NaN,0.5993,NaN,0.1316,NaN,NaN,NaN,-0.2116,NaN,0.2859,NaN,0.4149,NaN,NaN,NaN
ENSG00000000460.12,0.6636,-1.1962,0.0745,NaN,-0.1009,NaN,NaN,NaN,-0.4384,-0.2705,0.4664,NaN,0.6329,NaN,NaN,NaN


In [4]:
import gffutils
def gene_id_to_name(db):
    '''
    Returns a dictionary containing a gene_id:name translation
    Note: may be different if the 'gene_id' or 'gene_name' 
    keys are not in the source GTF file
    (taken from gscripts.region_helpers)
    '''
    genes = db.features_of_type('gene')
    gene_name_dict = {}
    for gene in genes:
        gene_id = gene.attributes['gene_id'][0] if type(gene.attributes['gene_id']) == list else gene.attributes[
            'gene_id']
        try:
            gene_name_dict[gene_id] = gene.attributes['gene_name'][0]
        except KeyError:
            print(gene.attributes.keys())
            print("Warning. Key not found for {}".format(gene))
            return 1
    return gene_name_dict

db_fn = '/projects/ps-yeolab/genomes/hg19/gencode_v19/gencode.v19.annotation.gtf.db'
db = gffutils.FeatureDB(db_fn)

id2name_dict = gene_id_to_name(db)

def convert_gene_id_to_name(df, id2name_dict):
    assert 'gene_name' not in df.columns
    df['gene_name'] = df.index.to_series().map(id2name_dict)
    df.set_index('gene_name', inplace=True)
    return df

In [5]:
convert_gene_id_to_name(merged, id2name_dict)

,rep_01|CDS_x,rep_01|5utr_x,rep_01|3utr_x,rep_01|5utr|3utr_x,rep_01|intron_x,rep_01|intergenic_x,rep_01|noncoding_exon_x,rep_01|noncoding_intron_x,rep_01|CDS_y,rep_01|5utr_y,rep_01|3utr_y,rep_01|5utr|3utr_y,rep_01|intron_y,rep_01|intergenic_y,rep_01|noncoding_exon_y,rep_01|noncoding_intron_y
gene_name,,,,,,,,,,,,,,,,
TSPAN6,3.6427,NaN,2.3706,NaN,0.6804,NaN,NaN,NaN,-3.2836,NaN,-1.9341,NaN,-0.8961,NaN,NaN,NaN
TNMD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DPM1,0.7458,NaN,2.6962,NaN,-0.6479,NaN,NaN,NaN,-1.7559,NaN,-1.5336,NaN,0.6446,NaN,NaN,NaN
SCYL3,NaN,NaN,0.5993,NaN,0.1316,NaN,NaN,NaN,-0.2116,NaN,0.2859,NaN,0.4149,NaN,NaN,NaN
C1orf112,0.6636,-1.1962,0.0745,NaN,-0.1009,NaN,NaN,NaN,-0.4384,-0.2705,0.4664,NaN,0.6329,NaN,NaN,NaN
FGR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CFH,-5.5446,NaN,NaN,NaN,-7.4321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FUCA2,1.3291,NaN,-0.2657,NaN,0.4520,NaN,NaN,NaN,-2.0291,-4.2340,-0.0900,NaN,0.0901,NaN,NaN,NaN
GCLC,0.1035,NaN,1.9413,NaN,0.0543,NaN,NaN,NaN,-0.1185,NaN,-0.9562,NaN,0.5729,NaN,NaN,NaN
